In [1]:
upstream = ['calculateNetworkStats', 'createTimeSeries'] # this means: execute raw.py, then clean.py
product = None

In [2]:
# Parameters
upstream = {"createTimeSeries": {"nb": "C:\\Users\\yosty\\Desktop\\Desktop_Folder\\14 - git\\timeSeriesDOTS\\ploomber\\dots\\00-data\\createTimeSeries.ipynb", "dotsTimeSeries": "C:\\Users\\yosty\\Desktop\\Desktop_Folder\\14 - git\\timeSeriesDOTS\\ploomber\\dots\\00-data\\clean\\dotsTimeSeries.csv"}, "calculateNetworkStats": {"nb": "C:\\Users\\yosty\\Desktop\\Desktop_Folder\\14 - git\\timeSeriesDOTS\\ploomber\\dots\\00-data\\calculateNetworkStats.ipynb", "DOTSnetStats": "C:\\Users\\yosty\\Desktop\\Desktop_Folder\\14 - git\\timeSeriesDOTS\\ploomber\\dots\\00-data\\clean\\DOTSnetStats.csv"}}
product = {"nb": "C:\\Users\\yosty\\Desktop\\Desktop_Folder\\14 - git\\timeSeriesDOTS\\ploomber\\dots\\01-timeSeries\\xgboost.ipynb"}


   In [previous work](https://rcyost.github.io/DOTS-network) I've calculated some basic network statistics on IMF Direction of Trade Statistics (DOTS) export data.

   I looked for relevant features using univariate linear regression [in this notebook](https://rcyost.github.io/network-feature-engineering-trade)

   In this notebook I'll use XGBoost.

  Table of Contents:
   1. Load and clean data
   2. For each trade series, XGBoost export series against the exporter's network statistics
      - This could be re-run on importer's statistics
      - Recalculate the network with edges as nodes: [example](https://youtu.be/p5LO97n3llg?t=235)
   3. Sort by mean absolute error.
   4. Collapse network statistics with PCA, repeat 2,3,4 on PCA series


   ### 1. Load and clean data

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


from numpy import absolute
from sklearn.metrics import mean_squared_error

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV


from tqdm import tqdm

from math import ceil

import pickle

c:\users\yosty\envs\python\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


c:\users\yosty\envs\python\lib\site-packages\statsmodels\compat\pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [4]:

timeSeries=(pd.read_csv(upstream['createTimeSeries']['dotsTimeSeries'])
    .pivot_table(index='period', columns=['ReferenceArea', 'CounterpartReferenceArea'], values='value')
)
# timeSeries=(pd.read_csv('dotsTimeSeries.csv')
#     .pivot_table(index='period', columns=['ReferenceArea', 'CounterpartReferenceArea'], values='value')
# )


tsPctChange=np.log(timeSeries).pct_change().iloc[1:].dropna(axis=1)
tsPctChange.columns=['-'.join(col) for col in tsPctChange.columns]
tsPctChange[tsPctChange>1.5]=np.nan
tsPctChange[tsPctChange<-1.5]=np.nan
tsPctChange=tsPctChange.dropna(axis=1)
tsPctChange.index=pd.to_datetime(tsPctChange.index)
tsPctChange=tsPctChange[tsPctChange.index > '1985-01-01']

netStats=pd.read_csv(upstream['calculateNetworkStats']['DOTSnetStats']).drop(['Unnamed: 0', 'CONNECTIVITY', 'HAS_BRIDGE', 'TOTAL_NET_VALUE', 'PAGERANK_NUMPY'],axis=1)
# netStats=pd.read_csv('DOTSnetStats.csv').drop(['Unnamed: 0', 'CONNECTIVITY', 'HAS_BRIDGE', 'TOTAL_NET_VALUE', 'PAGERANK_NUMPY'],axis=1)
netStats.set_index(['index', 'PERIOD'], inplace=True)
# get to period index and econ, stats cols
netStatsWide=(netStats
.reset_index()
.melt(id_vars=['index', 'PERIOD'])
.pivot_table(index='PERIOD', columns=['index', 'variable'], values='value')
)
netStatsWide.index = pd.to_datetime(netStatsWide.index)
netStatsWidePctChange=netStatsWide.pct_change().iloc[1:].dropna(axis=1)
netStatsWidePctChange.index=pd.to_datetime(netStatsWidePctChange.index)
netStatsWidePctChange=netStatsWidePctChange[netStatsWidePctChange.index > '1985-01-01']

In [5]:

# lag the net stats to not leak information
netStatsWidePctChange=netStatsWidePctChange.shift(-1).iloc[:-1]
# take off a period of time series so sizes match
tsPctChange=tsPctChange.iloc[:-1]

In [6]:
netStats.corr()

,DEGREE,IN_DEGREE,OUT_DEGREE,DEGREE_CENTRALITY,IN_DEGREE_CENTRALITY,OUT_DEGREE_CENTRALITY,AVG_NEIGHBOR_DEGREE,PAGERANK,KATZ,CLOSENESS_CENTRALITY,BETWEENNESS_CENTRALITY,CLUSTCOEF,NUM_NODES,NUM_EDGES,AVERAGECLUSTCOEF,TRIANGLES
DEGREE,1.000000,0.962988,0.977184,0.992434,0.963476,0.955832,0.493826,0.406518,-0.132791,0.835492,0.555967,-0.498091,0.613884,0.631775,-0.188338,0.963967
IN_DEGREE,0.962988,1.000000,0.883767,0.941548,0.993526,0.847207,0.520920,0.371575,-0.139517,0.887069,0.473481,-0.395510,0.676223,0.695930,-0.207463,0.955192
OUT_DEGREE,0.977184,0.883767,1.000000,0.980944,0.889714,0.991741,0.446807,0.412921,-0.120589,0.751422,0.592064,-0.553033,0.532852,0.548381,-0.163477,0.920773
DEGREE_CENTRALITY,0.992434,0.941548,0.980944,1.000000,0.955052,0.974949,0.451287,0.453122,-0.149922,0.806000,0.621666,-0.556223,0.546003,0.562473,-0.190481,0.929922
IN_DEGREE_CENTRALITY,0.963476,0.993526,0.889714,0.955052,1.000000,0.865192,0.479139,0.419466,-0.159216,0.871295,0.534531,-0.442852,0.615455,0.634020,-0.214711,0.933060
OUT_DEGREE_CENTRALITY,0.955832,0.847207,0.991741,0.974949,0.865192,1.000000,0.403864,0.451742,-0.134137,0.709634,0.650507,-0.608596,0.461798,0.475728,-0.161105,0.872909
AVG_NEIGHBOR_DEGREE,0.493826,0.520920,0.446807,0.451287,0.479139,0.403864,1.000000,-0.122871,-0.043899,0.699419,-0.031501,0.006313,0.775069,0.797217,-0.337722,0.489305
PAGERANK,0.406518,0.371575,0.412921,0.453122,0.419466,0.451742,-0.122871,1.000000,-0.040801,0.222944,0.763224,-0.448343,-0.061415,-0.058331,0.012322,0.290918
KATZ,-0.132791,-0.139517,-0.120589,-0.149922,-0.159216,-0.134137,-0.043899,-0.040801,1.000000,-0.085656,-0.075148,0.090840,-0.019724,-0.016766,0.017145,-0.097818
CLOSENESS_CENTRALITY,0.835492,0.887069,0.751422,0.806000,0.871295,0.709634,0.699419,0.222944,-0.085656,1.000000,0.347980,-0.275433,0.817167,0.838388,-0.469232,0.818362


In [7]:
netStatsWidePctChange.head()

index           Afghanistan                                           \
variable   AVERAGECLUSTCOEF CLOSENESS_CENTRALITY CLUSTCOEF    DEGREE   
PERIOD                                                                 
1985-02-01         0.002301             0.000000 -0.007292  0.052632   
1985-03-01         0.011363             0.006944  0.023946 -0.083333   
1985-04-01        -0.000926             0.003484  0.017832  0.036364   
1985-05-01        -0.008471            -0.020478 -0.013485 -0.070175   
1985-06-01         0.005615             0.017361 -0.024007  0.094340   

index                                                                  \
variable   DEGREE_CENTRALITY IN_DEGREE IN_DEGREE_CENTRALITY      KATZ   
PERIOD                                                                  
1985-02-01          0.052632  0.034483             0.034483 -0.298043   
1985-03-01         -0.083333  0.000000             0.000000  0.968943   
1985-04-01          0.036364  0.033333             0.033333  0.078214   
1985-05-01         -0.070175 -0.161290            -0.161290 -0.125089   
1985-06-01          0.094340  0.192308             0.192308 -0.114136   

index                           ... Yemen, P.D. Rep.            \
variable   NUM_EDGES NUM_NODES  ...        CLUSTCOEF    DEGREE   
PERIOD                          ...                              
1985-02-01  0.009121       0.0  ...        -0.002829  0.145455   
1985-03-01  0.001928       0.0  ...         0.003962 -0.047619   
1985-04-01  0.007457       0.0  ...         0.001344  0.033333   
1985-05-01 -0.007880       0.0  ...         0.005263 -0.064516   
1985-06-01  0.010108       0.0  ...         0.007962  0.051724   

index                                                                  \
variable   DEGREE_CENTRALITY IN_DEGREE IN_DEGREE_CENTRALITY      KATZ   
PERIOD                                                                  
1985-02-01          0.145455  0.189189             0.189189  0.682972   
1985-03-01         -0.047619 -0.068182            -0.068182 -0.108051   
1985-04-01          0.033333  0.048780             0.048780 -0.359584   
1985-05-01         -0.064516 -0.093023            -0.093023 -0.336619   
1985-06-01          0.051724  0.051282             0.051282  0.441468   

index                                               
variable   NUM_EDGES NUM_NODES  PAGERANK TRIANGLES  
PERIOD                                              
1985-02-01  0.009121       0.0  0.083911  0.316947  
1985-03-01  0.001928       0.0 -0.274875 -0.085462  
1985-04-01  0.007457       0.0  0.049424  0.105263  
1985-05-01 -0.007880       0.0  0.207353 -0.168124  
1985-06-01  0.010108       0.0  0.156956  0.089953  

[5 rows x 1624 columns]

In [8]:
netStatsWidePctChange.corr()

index                                      Afghanistan                       \
variable                              AVERAGECLUSTCOEF CLOSENESS_CENTRALITY   
index            variable                                                     
Afghanistan      AVERAGECLUSTCOEF             1.000000             0.073669   
                 CLOSENESS_CENTRALITY         0.073669             1.000000   
                 CLUSTCOEF                    0.413637            -0.018438   
                 DEGREE                       0.061910             0.508878   
                 DEGREE_CENTRALITY            0.076437             0.509873   
...                                                ...                  ...   
Yemen, P.D. Rep. KATZ                        -0.041185             0.018928   
                 NUM_EDGES                    0.284493             0.120589   
                 NUM_NODES                   -0.036407            -0.128162   
                 PAGERANK                    -0.092208            -0.032935   
                 TRIANGLES                    0.007055             0.029118   

index                                                                        \
variable                              CLUSTCOEF    DEGREE DEGREE_CENTRALITY   
index            variable                                                     
Afghanistan      AVERAGECLUSTCOEF      0.413637  0.061910          0.076437   
                 CLOSENESS_CENTRALITY -0.018438  0.508878          0.509873   
                 CLUSTCOEF             1.000000 -0.283062         -0.264100   
                 DEGREE               -0.283062  1.000000          0.996922   
                 DEGREE_CENTRALITY    -0.264100  0.996922          1.000000   
...                                         ...       ...               ...   
Yemen, P.D. Rep. KATZ                 -0.054971  0.005986          0.005765   
                 NUM_EDGES             0.112460  0.205111          0.207166   
                 NUM_NODES            -0.020927 -0.046906         -0.050575   
                 PAGERANK             -0.109745  0.061522          0.060699   
                 TRIANGLES             0.018873  0.000550          0.000540   

index                                                                 \
variable                              IN_DEGREE IN_DEGREE_CENTRALITY   
index            variable                                              
Afghanistan      AVERAGECLUSTCOEF     -0.016576             0.002934   
                 CLOSENESS_CENTRALITY  0.852449             0.857545   
                 CLUSTCOEF            -0.148596            -0.127168   
                 DEGREE                0.644623             0.645099   
                 DEGREE_CENTRALITY     0.635419             0.642452   
...                                         ...                  ...   
Yemen, P.D. Rep. KATZ                  0.011210             0.011045   
                 NUM_EDGES             0.145586             0.148514   
                 NUM_NODES            -0.064192            -0.068557   
                 PAGERANK              0.023219             0.022310   
                 TRIANGLES             0.098578             0.099746   

index                                                                ...  \
variable                                   KATZ NUM_EDGES NUM_NODES  ...   
index            variable                                            ...   
Afghanistan      AVERAGECLUSTCOEF      0.038123  0.315566 -0.156818  ...   
                 CLOSENESS_CENTRALITY  0.097872  0.336982  0.041482  ...   
                 CLUSTCOEF             0.032734  0.007868 -0.214551  ...   
                 DEGREE                0.010131  0.269504  0.067043  ...   
                 DEGREE_CENTRALITY     0.010168  0.231666 -0.010925  ...   
...                                         ...       ...       ...  ...   
Yemen, P.D. Rep. KATZ                 -0.025253 -0.025512  0.003067  ...   
                 

In [9]:
tsPctChange.head()

,Argentina-Brazil,Argentina-Chile,Argentina-Japan,Australia-Canada,Australia-China,Australia-France,Australia-Germany,Australia-India,Australia-Italy,Australia-Japan,...,United States-Switzerland,"United States-Taiwan, Province of China",United States-Thailand,United States-Trinidad and Tobago,United States-Turkey,United States-United Arab Emirates,United States-United Kingdom,United States-Uruguay,"United States-Venezuela, Bolivarian Republic",Uruguay-Germany
period,,,,,,,,,,,,,,,,,,,,,
1985-02-01,0.062522,-0.154539,-0.293125,0.142470,0.062065,-0.097768,0.065731,-0.131695,0.000170,-0.004596,...,-0.009254,-0.011090,-0.006611,-0.090844,-0.027302,0.038621,-0.008302,-0.067725,0.009637,0.028158
1985-03-01,-0.157772,-0.028198,0.374053,0.018934,-0.088413,-0.003325,-0.037304,0.080223,-0.137057,-0.023654,...,0.066524,0.014933,0.085342,0.075829,-0.105616,0.006352,0.053685,0.174717,-0.005320,-0.231935
1985-04-01,0.076867,0.130921,0.230518,-0.087285,0.164218,0.056872,-0.091518,0.128781,0.182579,0.051857,...,-0.061728,-0.040338,-0.094513,-0.015796,0.235700,-0.085926,-0.052472,-0.041805,0.017312,0.447250
1985-05-01,-0.079725,-0.085492,-0.072008,0.074769,0.035272,-0.029843,0.165752,0.010152,0.004218,-0.000145,...,0.021356,0.009626,-0.042600,0.032492,-0.043499,-0.008756,-0.004399,0.122381,0.037731,0.028077
1985-06-01,0.037981,0.148035,0.147914,0.027766,0.093546,0.115701,-0.122772,0.011665,-0.078161,0.012099,...,-0.016160,0.010196,-0.013965,-0.026642,-0.039266,0.012857,-0.013124,-0.119213,-0.034192,0.095110


In [10]:
importers=pd.Series(col.split('-')[0] for col in tsPctChange.columns).unique()
exporters=pd.Series(col.split('-')[1] for col in tsPctChange.columns).unique()
allEcons=sorted(set(list(importers) + list(exporters)))
netStats=pd.Series(col[1] for col in netStatsWidePctChange.columns).nunique()

print('The upper-bound on number of tests:', len(allEcons)*netStats)

The upper-bound on number of tests: 1328


  ## 2. Loop and XGBoost

In [11]:

# https://www.kaggle.com/felipefiorini/xgboost-hyper-parameter-tuning
# https://www.kaggle.com/felipefiorini/xgboost-hyper-parameter-tuning/notebook

def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
    }

    xgb_model = xgb.XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 0)

    gsearch.fit(X_train,y_train)

    return(gsearch.best_params_)

In [12]:

# https://xgboost.readthedocs.io/en/latest/python/examples/index.html
# https://xgboost.readthedocs.io/en/stable/parameter.html
# https://www.kaggle.com/stuarthallows/using-xgboost-with-scikit-learn

# https://www.kaggle.com/prashant111/a-guide-on-xgboost-hyperparameters-tuning

results={}

econs=pd.Series(col for col in tsPctChange.columns).unique()
tempSeries='Argentina-Brazil'

# create dataset
# network statistics
X=netStatsWidePctChange[[col for col in netStatsWidePctChange.columns if col[0] == tempSeries.split('-')[0] or col[0] == tempSeries.split('-')[1]]]
X.columns=["-".join(col) for col in X.columns]
X_temp=X

# bilateral trade series
y=tsPctChange[[tempSeries]]

# if there is data for model
if not X_temp.empty and not y.empty:
    results[tempSeries]={}
    results[tempSeries]['y_std']=y.std()
    results[tempSeries]['series']=tempSeries
    X_train, X_test, y_train, y_test = train_test_split(X_temp, y, test_size=0.1, shuffle=False)
    results[tempSeries]['y_test_std']=y_test.std()

    #bestParams=hyperParameterTuning(X_train, y_train)

    bst = xgb.XGBRegressor(
        objective = 'reg:squarederror',
        colsample_bytree = .05,
        learning_rate = .01,
        max_depth = 3,
        min_child_weight = 5,
        n_estimators = 500,
        subsample = .5,
        nthread=4)

    #results[tempSeries]['bestParams']=bestParams

    bst.fit(X_train, y_train)

    results[tempSeries]['model']=bst

    y_pred = bst.predict(X_test)

    mse=mean_squared_error(y_test, y_pred)
    results[tempSeries]['mse']=mse

    results[tempSeries]['data']=[X_train, X_test, y_train, y_test, y_pred]


    importances=['weight', 'gain', 'cover']
    for importance in importances:
        results[tempSeries][importance]=(bst.get_booster().get_score(importance_type=importance))